# Построение движка для игр на pygame - урок 5

## Повторение

Опишем колизию между персонажем и границами экрана, так что бы персонаж не мог выйти за граници экрана:

In [ ]:
from first_engine.game import Game
from first_engine.objects import Object


class FirstGame(Game):

    def run(self):

        hero = Object(parent=self.parentSurface, speed=4)

        while self.RUNNER:
            self.fps_counter(60)
            self.display_update()
            self.window_fill()

            hero.blit()
            hero.motion()
            
            # тут все очень тривиально
            if hero.body.bottom > self.parentBody.bottom: # если низ персонажа ниже нижней части экрана
                hero.body.bottom = self.parentBody.bottom # то низ персонажа становится равен низу экрана
            elif hero.body.top < self.parentBody.top: # иначе если верх персонажа выше верхней части экрана
                hero.body.top = self.parentBody.top # то верх персонажа становится равен верху экрана
            if hero.body.right > self.parentBody.right:
                hero.body.right = self.parentBody.right # и так далее...
            elif hero.body.left < self.parentBody.left:
                hero.body.left = self.parentBody.left

            for event in self.events():
                # print(event)  # отслеживание событий
                self.close(event)


FirstGame().run()

Добавим этот метод в класс __Game__

In [ ]:
"""
Модуль с основным классом-конструктором Game - для создания игры.
"""

import pygame


class Game:
    ...

    def __init__(self, width=400, height=300, color=(0, 0, 0)):
        ...

    @staticmethod
    def display_update():
        ...

    @staticmethod
    def events():
        ...

    def window_fill(self):
        ...
 
    def fps_counter(self, FPS=30) -> None:
        ...

    def close(self, event) -> None:
        ...

    def window_borders(self, obj): # экран преграждает путь какому либо объекту, что бы он не вышел за границы
        if obj.body.bottom > self.parentBody.bottom:
            obj.body.bottom = self.parentBody.bottom
        elif obj.body.top < self.parentBody.top:
            obj.body.top = self.parentBody.top
        if obj.body.right > self.parentBody.right:
            obj.body.right = self.parentBody.right
        elif obj.body.left < self.parentBody.left:
            obj.body.left = self.parentBody.left

Обработать выход одного объекта за пределы другого не сложно, но что если, мы хотим, что бы 2 объекта не могли между собой пересечься?

Давайте попробуем:

In [ ]:
from first_engine.game import Game
from first_engine.objects import Object


class FirstGame(Game):

    def run(self):

        hero = Object(parent=self.parentSurface, speed=4) # создаем перса
        wall = Object(self.parentSurface, 
                      x=200, y=100, width=100, height=100, 
                      color=(255, 100, 100)) # создаем преграду

        while self.RUNNER:
            self.fps_counter(60)
            self.display_update()
            self.window_fill()

            hero.blit() # отрисовываем героя
            hero.motion() # даем ему возможность перемещаться

            wall.blit() # отрисовываем преграду

            if hero.body.right > wall.body.left: # если правая часть перса нахобится дальше левой части преграды
                hero.body.right = wall.body.left # то правая часть перса становится равна левой части преграды

            for event in self.events():
                # print(event)
                self.close(event)


FirstGame().run()

Можно было бы продолжить, но здесь мы столкнулись с проблемой, герой вообще не может перейти дальше левой части преграды даже если поднимется выше или ниже преграды

Здесь нам нужно обрабатывать колизию между двумя созданными объектами, то есть, когда 1 объект входит в тело другого, то мы получаем сигнал и только после этого обрабатываем свойство преграды

# Задание

Постройте логику так, что бы  исправить баг с невидимой преградой